# Question Augmentation (similarity)

A frequent source of difficulty in using Translator Knowledge Graphs is the traversal of similar nodes.  If two non-identical (but very similar) concepts exist, they may be independently associated with other information by individual KPs.  If a user is unaware of this subtlety, it is easy to miss highly relevant answers.

Here is a simple query: (gene)--(chemical)--(disease).  This query might be used to explain gene/disease relationships that are driven by some metabolic process.   For instance, there is a known association between the gene SLC13A2 and Ulcerative Colitis:

In [38]:
class RobokopMessenger:
    def __init__(self):
        self.url = 'http://robokop.renci.org:4868'
    def pipeline(self,request,yank = True):
        #normalize question
        response = requests.post( f'{self.url}/normalize', json=request )
        normalized = response.json()
        #answer question
        request = { 'message': normalized, }
        response = requests.post( f'{self.url}/answer', json=request )
        answered = response.json()
        if not yank:
            return answered
        #Yank
        request = { 'message': answered, }
        response = requests.post( f'{self.url}/yank', json=request )
        filled = response.json()
        return filled
    def get_links_for(self,curie,stype):
        query = { 'nodes': [{'id': 'n0', 'curie':curie, 'type': stype},
                            {'id': 'n1'}],
                  'edges': [{'id': 'e0', 'source_id': 'n0', 'target_id': 'n1'}]}
        request = { "message": { "query_graph": query } }
        result = self.pipeline(request)
        
def get_view_url(returnanswer,robokop='robokop.renci.org'):
    """Given an answer in KGS v0.9 format, post the answer to robokop, and return a link that can be followed to
    view the answer in the UI"""
    view_post_url = f'http://{robokop}/api/simple/view/'
    view_post_response = requests.post(view_post_url, json=returnanswer)
    uid=view_post_response.json()
    view_url = f'http://{robokop}/simple/view/{uid}'
    return view_url

In [44]:
import requests
import json

question = { 'nodes': [{'id':'n0', 'type':'gene', 'curie':'NCBIGene:5443'},
                      {'id': 'n1', 'type':'disease'}],
             'edges': [ {'id': 'e0', 'source_id': 'n1', 'target_id': 'n0', 'type':'contributes_to'}]}
message = {'message': {'query_graph': question}}

#vm = {'message':{'query_graph':question, 'knowledge_graph':{}, 'results':[]}}
#url = get_view_url(vm)
#print(url)

strider = 'http://robokop.renci.org:5781/query'
response = requests.post(strider,json=message)
if response.status_code == 200:
    pid = response.json()
    r = requests.get('http://robokop.renci.org:5781/results',params={'query_id':pid})
    print(json.dumps(r.json(),indent=2))
else:
    print(response.status_code)



[]


In [16]:
rm = RobokopMessenger()
result = rm.pipeline(message, yank=True)
print(result)

{'knowledge_graph': {'edges': [{'ctime': [1573152307.1818748, 1572941363.5901597], 'edge_source': ['pharos.disease_get_gene', 'pharos.gene_get_disease'], 'id': 'e6acbff807fd74bcd891b32ca7c11f6f', 'predicate_id': 'NCIT:R176', 'publications': [], 'relation': ['PHAROS:gene_involved', 'PHAROS:gene_involved'], 'relation_label': ['gene_involved', 'gene_involved'], 'source_database': ['pharos', 'pharos'], 'source_id': 'MONDO:0005090', 'target_id': 'HGNC:9201', 'type': 'disease_to_gene_association'}], 'nodes': [{'equivalent_identifiers': ['MEDDRA:10039642', 'DOID:5419', 'MEDDRA:10039632', 'MEDDRA:10012297', 'MEDDRA:10039626', 'MESH:D012559', 'UMLS:C0036341', 'MEDDRA:10046156', 'MONDO:0005090', 'MEDDRA:10046150', 'EFO:0000692'], 'id': 'MONDO:0005090', 'name': 'Schizophrenia', 'psychiatric_disorder': True, 'synonyms': ['schizophrenia', 'schizophrenia-1', 'schizoaffective disorder', 'schizophrenia with or without an affective disorder', 'schizophrenia; SCZD', 'SCZD'], 'type': ['disease', 'disease

And I can ask for chemicals that relate the gene and disease:

In [17]:
question = { 'nodes': [{'id':'n0', 'type':'gene', 'curie':'NCBIGene:5443'},
                       {'id':'n1', 'type':'chemical_substance'},
                      {'id': 'n2', 'type':'disease', 'curie':'MONDO:0005090'}],
             'edges': [ {'id': 'e0', 'source_id': 'n0', 'target_id': 'n1'},
                        {'id': 'e1', 'source_id': 'n1', 'target_id': 'n2'}]}
message = {'message': {'query_graph': question}}

strider = 'http://robokop.renci.org:5781/query'
response = requests.post(strider,json=message)
print(response.status_code)

500
